In [1]:
# Video
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Define video function
def create_video(pipeline_in, input_video, output_video):
    # Process video
    video = VideoFileClip( input_video )
    video_processed = video.fl_image( pipeline_in )
    %time video_processed.write_videofile( output_video, audio=False )

# Pipeline
def pipeline( image ):

    return image

# Create video
create_video(pipeline, "a.mov", "output.mp4" )

# Show video
HTML("""<video width="960" height="540" controls><source src="{0}"></video>""".format( "output.mp4" ))



[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


100%|██████████| 829/829 [00:26<00:00, 28.97it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 

CPU times: user 1.72 s, sys: 1.02 s, total: 2.74 s
Wall time: 28 s
